In [14]:
import csv
import io
import os
import time
import uuid 
import re
import pandas as pd
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

In [2]:
df = pd.read_csv('AWSDocs.csv')

In [3]:
CHROMA_DB_PATH = "./chroma_db_AWSDocs"
EMBEDDING_MODEL_NAME = "nomic-embed-text" 
BATCH_SIZE = 50  
CHUNK_SIZE = 1000 
CHUNK_OVERLAP = 200

In [4]:
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)

In [5]:
vector_store = Chroma(
    collection_name="AWSDocs",
    persist_directory=CHROMA_DB_PATH,
    embedding_function=embeddings
)

2025-11-11 22:53:15,653 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [6]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

In [ ]:
# loader = DoclingLoader(
#     file_path='https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf',
#     export_type=ExportType.MARKDOWN,
# )

In [19]:
for row in df.iterrows():
    url = row[1]['PDF_URL']
    domain = row[1]['Domain']
    service = row[1]['Service']
    if not url:
        continue
    else:
        try:
            loader = DoclingLoader(
                file_path=url,
                export_type=ExportType.MARKDOWN,
            )
            docs_as_markdown = loader.load()
            markdown_content = docs_as_markdown[0].page_content
            semantic_chunks = markdown_splitter.split_text(markdown_content)

            for chunk in semantic_chunks:
                chunk.metadata["domain"] = domain
                chunk.metadata["service"] = service
                chunk.metadata["source"] = url

            final_chunks = text_splitter.split_documents(semantic_chunks)

            if final_chunks:
                print(f"Found {len(final_chunks)} final chunks to add.") 

            chunk_ids = [str(uuid.uuid4()) for _ in final_chunks]

            for i in range(0, len(final_chunks)):
        
                # Get the current chunk
                chunk = final_chunks[i]
                
                h1 = chunk.metadata.get("Header 1", "")
                h2 = chunk.metadata.get("Header 2", "")
                h3 = chunk.metadata.get("Header 3", "")
                
                is_toc = 'table of contents' in h1.lower() or \
                        'table of contents' in h2.lower() or \
                        'table of contents' in h3.lower()

                content_to_check = chunk.page_content.strip() # Remove leading/trailing spaces
                is_table_junk = False
                if content_to_check: # Only check if the string is not empty
                    is_table_junk = bool(re.fullmatch(r'[|\-\s]+', content_to_check))
                
                
                if is_toc or is_table_junk:
                    if is_toc:
                        print(f"Skipping chunk {i+1} (Table of Contents)")
                    if is_table_junk:
                        print(f"Skipping chunk {i+1} (Markdown table fragment)")
                    pass # Skip this chunk
                
                else:
                    batch_docs = [chunk] # Get the chunk as a list
                    batch_ids = [chunk_ids[i]] # Get the corresponding ID
                    
                    batch_num = i + 1
                    total_batches = len(final_chunks)

                    print(f"  Adding batch {batch_num}/{total_batches}...")
                    
                    try:
                        vector_store.add_documents(
                            documents=batch_docs,
                            ids=batch_ids
                        )
                    except Exception as e:
                        print(f"Error adding batch {batch_num}/{total_batches}: {e}")
                        continue 

        except Exception as e:
            print(f"Error processing {url}: {e}")
            continue


2025-11-11 23:06:17,487 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-11 23:06:17,829 - INFO - Going to convert document batch...
2025-11-11 23:06:17,831 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-11 23:06:17,895 - INFO - Loading plugin 'docling_defaults'
2025-11-11 23:06:17,906 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-11 23:06:17,907 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-11 23:06:17,917 - INFO - Loading plugin 'docling_defaults'
2025-11-11 23:06:17,934 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-11 23:06:17,934 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-11 23:06:18,970 - INFO - Auto OCR model selected ocrmac.
2025-11-11 23

Found 10887 final chunks to add.
  Adding batch 1/10887...


2025-11-11 23:25:15,879 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:15,944 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:15,997 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:16,054 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:16,084 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:16,102 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Error adding batch 1/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 2/10887...
Error adding batch 2/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 3/10887...
Error adding batch 3/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
Skipping chunk 4 (Table of Contents)
Skipping chunk 5 (Table of Contents)
Skipping chunk 6 (Table of Contents)
Skipping chunk 7 (Table of Contents)
Skipping chunk 8 (Table of Contents)
Skipping chunk 9 (Table of Contents)
Skipping chunk 10 (Table of Contents)
Skipping chunk 11 (Table of Contents)
Skipping chunk 12 (Table of Contents)
Skipping chunk 13 (Table of Contents)
Skipping chunk 14 (Table of Contents)
Skipping chunk 15 (Table of Contents)
Skipping chunk 16 (Table of Contents)
Skipping chunk 17 (Table of Contents)
Sk

2025-11-11 23:25:16,123 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:16,144 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:16,163 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:16,184 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:16,215 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:16,247 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:25:16,269 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


...

Error adding batch 10875/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 10876/10887...
Error adding batch 10876/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
Skipping chunk 10877 (Markdown table fragment)
  Adding batch 10878/10887...
Error adding batch 10878/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 10879/10887...
Error adding batch 10879/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 10880/10887...
Error adding batch 10880/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 10881/10887...
Error adding batch 10881/10887: Error getting collection: Database error: error ret

2025-11-11 23:33:22,781 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:33:22,811 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:33:22,837 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:33:22,894 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:33:22,926 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 23:33:22,953 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Error adding batch 10882/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 10883/10887...
Error adding batch 10883/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 10884/10887...
Error adding batch 10884/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 10885/10887...
Error adding batch 10885/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 10886/10887...
Error adding batch 10886/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file
  Adding batch 10887/10887...
Error adding batch 10887/10887: Error getting collection: Database error: error returned from database: (code: 14) unable to open 

2025-11-11 23:33:25,930 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-11 23:33:26,104 - INFO - Going to convert document batch...
2025-11-11 23:33:26,106 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-11 23:33:26,109 - INFO - Auto OCR model selected ocrmac.
2025-11-11 23:33:26,112 - INFO - Accelerator device: 'mps'
2025-11-11 23:33:34,246 - INFO - Accelerator device: 'mps'
2025-11-11 23:33:36,153 - INFO - Processing document lambda-dg.pdf


: 